## 필요 라이브러리 호출

In [228]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import time

## 데이터 크롤링

In [108]:
# 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
# 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
# webdriver 설치 위치를 정의한다.
browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')

def crawling(url):
    # 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
    # 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
    # webdriver 설치 위치를 정의한다.
    browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')
    # 불러오고자하는 url을 .get으로 호출한다.
    browser.get(url)
    
    # page_source파라미터를 이용하면 HTML정보를 가지고 온다.
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')   # 뷰티풀숩으로 HTML을 파싱하고 필요한 데이터 수집
    table_tags = soup.find_all("table")   # find_all함수를 이용하면, TABLE태그로 지정된 곳만 뽑아서, 배열 형태로 저장
    table = table_tags[0]    # html상에서 table 순서, 첫번째 테이블 가지고 와야 함.
    p=parser.make2d(table)

    df_total=pd.DataFrame(p[1:],columns=p[0])    # 데이터 프레임으로 저장

    num = soup.find_all("button")[5].text   # 5번째 button에 페이지수 정보가 있음
    for j in range(1, int(num)):
        btn = browser.find_element_by_class_name('paginationWidget-next')
        btn.click()     # 버튼 클릭
        time.sleep(5)

        # 2페이지 이후 데이터 병합
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table_tags = soup.find_all("table")
        table = table_tags[0]
        p = parser.make2d(table)

        df = pd.DataFrame(p[1:], columns=p[0])
        df_total = pd.concat([df_total, df], 0)
        
    return df_total
        # 엑셀로 다운로드
        # df_total.to_excel("test.xlsx")

In [109]:
list_url = ["http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+hitting&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=hitting&page=1&ts=1580532864437&playerType=ALL",
"http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+fielding&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=fielding&page=1&ts=1580539033522&playerType=ALL"]

df_total_hit = crawling(list_url[0])
df_total_field = crawling(list_url[1])

In [113]:
df_total_hit

,RK,Player,Team,,▲,Pos,G▼,AB▼,R▼,H▼,...,SAC▼,SF▼,TB▼,XBH▼,GDP▼,GO▼,AO▼,GO_AO▼,NP▼,PA▼
0,1,"Alaniz, R",CIN,,595798,P,8,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
1,1,"Brebbia, J",STL,,605154,P,63,1,1,1,...,0,0,1,0,0,0,0,-.--,1,1
2,1,"Drake, O",TB,,543118,P,3,1,0,1,...,0,0,1,0,0,0,0,-.--,4,1
3,1,"Liriano, F",PIT,,434538,P,67,1,0,1,...,0,0,1,0,0,0,0,-.--,15,2
4,1,"Lugo, S",NYM,,607625,P,57,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,984,"Yates, K",SD,,489446,P,59,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
33,984,"Ynoa, G",BAL,,593679,P,4,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
34,984,"Ynoa, H",ATL,,660623,P,2,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
35,984,"Zamora, D",NYM,,623354,P,17,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0


In [111]:
df_total_field

,RK,Player,Team,,▲,Pos,G▼,GS▼,INN▼,TC▼,...,A▼,E▼,DP▼,SB▼,CS▼,SBPCT▲,PB▼,C_WP▼,FPCT▼,RF▼
0,1,"Abreu, B",HOU,,650556,P,7,0,8.2,4,...,2,0,0,-,-,-,-,-,1.000,0.57
1,1,"Adames, C",SF,,542436,SS,7,5,46.0,15,...,8,0,2,-,-,-,-,-,1.000,2.14
2,1,"Adams, A",DET,,542866,P,15,0,16.2,4,...,3,0,0,-,-,-,-,-,1.000,0.27
3,1,"Adams, C",NYY,,664856,P,13,0,25.1,2,...,2,0,0,-,-,-,-,-,1.000,0.15
4,1,"Adduci, J",CHC,,451192,RF,1,1,8.0,5,...,0,0,0,-,-,-,-,-,1.000,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1319,"Walker, T",ARI,,592836,P,1,1,1.0,0,...,0,0,0,-,-,-,-,-,-,0.00
1,1319,"Warren, A",SEA,,605521,P,6,0,5.1,0,...,0,0,0,-,-,-,-,-,-,0.00
2,1319,"Williams, A",WSH,,657117,P,2,0,0.1,0,...,0,0,0,-,-,-,-,-,-,0.00
3,1319,"Williams, D",MIL,,642207,P,13,0,13.2,0,...,0,0,0,-,-,-,-,-,-,0.00


## 전처리 진행
- 불필요한 컬럼 삭제(G, GS) , RK, Team, 빈 컬럼, 등
- hit & field data mergy : key column Player & Team $ Position
- 1B, 2B, 3B, SS 포지션만 추출
- AB 타석수(20), INN(20)가 일정 값 이상의 선수만 대상으로 한다.


In [148]:
# 컬럼 rename
df_total_hit = df_total_hit.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', 
                                            '':'del', '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'AB▼':'AB', 'R▼':'R',
                                            'H▼':'H', '2B▼':'2B', '3B▼':'3B', 'HR▼':'HR',
                                            'RBI▼':'RBI', 'BB▼':'BB', 'SO▼':'SO', 'SB▼':'SB',
                                            'CS▼':'CS', 'AVG▼':'AVG', 'OBP▼':'OBP',
                                            'SLG▼':'SLG', 'OPS▼':'OPS', 'IBB▼':'IBB',
                                            'HBP▼':'HBP', 'SAC▼':'SAC', 'SF▼':'SF', 
                                            'TB▼':'TB', 'XBH▼':'XBH', 'GDP▼':'GDP',
                                            'GO▼':'GO', 'AO▼':'AO', 'GO_AO▼':'GO_AO',
                                            'NP▼':'NP', 'PA▼':'PA'})
df_total_field = df_total_field.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', '':'del',
                                            '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'GS▼':'GS',
                                            'INN▼':'INN', 'TC▼':'TC', 'PO▼':'PO', 'A▼':'A',
                                            'E▼':'E', 'DP▼':'DP', 'SB▼':'SB', 'CS▼':'CS',
                                            'SBPCT▲':'SBPCT', 'PB▼':'PB', 'C_WP▼':'C_WP',
                                            'FPCT▼':'FPCT', 'RF▼':'RF'})

Index(['RK', 'Player', 'Team', 'del', 'del2', 'Pos', 'G', 'AB', 'R', 'H', '2B',
       '3B', 'HR', 'RBI', 'BB', 'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS',
       'IBB', 'HBP', 'SAC', 'SF', 'TB', 'XBH', 'GDP', 'GO', 'AO', 'GO_AO',
       'NP', 'PA'],
      dtype='object')

In [333]:
# 데이터 백업

player_hit_data = df_total_hit
player_field_data = df_total_field

In [334]:
# 불필요한 컬럼 삭제

player_hit_data = player_hit_data.drop(['RK', 'del', 'del2', 'GO_AO', 'G'], axis=1)
player_field_data = player_field_data.drop(['RK', 'del', 'del2', 'G', 'GS', 'SB', 'CS', 'SBPCT','PB','C_WP'], axis=1)

In [335]:
# column Player & Team $ Position 기준 병합

df_total = pd.merge(player_hit_data, player_field_data, on=['Player', 'Team', 'Pos']) # how='outer')
df_total.columns
#df_total.to_excel("test.xlsx")

Index(['Player', 'Team', 'Pos', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB',
       'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'IBB', 'HBP', 'SAC', 'SF',
       'TB', 'XBH', 'GDP', 'GO', 'AO', 'NP', 'PA', 'INN', 'TC', 'PO', 'A', 'E',
       'DP', 'FPCT', 'RF'],
      dtype='object')

In [336]:
# 컬럼 조건문으로 행 추출하기 Position 컬럼에서 "1B", "2B", "3B", "SS"

# 하나의 조건문이라면 "해당 컬럼 == 하나조건"으로 하면 되지만, "해당컬럼.isin([여러조건])"
# df_total_hit_infld = df_total_hit[df_total_hit['Pos'] == "2B"]
df_total_infld = df_total[df_total['Pos'].isin(["1B", "2B", "3B", "SS"])]
df_total_infld

,Player,Team,Pos,AB,R,H,2B,3B,HR,RBI,...,NP,PA,INN,TC,PO,A,E,DP,FPCT,RF
12,"Colon, C",CIN,2B,6,1,3,0,0,0,1,...,28,8,3.2,1,0,1,0,0,1.000,0.33
22,"Kendrick, H",WSH,3B,334,61,115,23,1,17,62,...,1397,370,601.0,401,308,91,2,35,.995,4.29
23,"Anderson, T",CWS,SS,498,81,167,32,0,18,56,...,1765,518,1050.0,532,182,324,26,75,.951,4.11
24,"Arraez, L",MIN,2B,326,54,109,20,1,4,28,...,1486,366,716.0,227,101,121,5,31,.978,2.22
37,"Solano, D",SF,SS,215,27,71,13,1,4,23,...,903,228,407.0,199,84,113,2,27,.990,3.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,"Stamets, E",CLE,SS,41,4,2,1,0,0,2,...,192,48,118.0,40,10,28,2,3,.950,2.53
836,"Haggerty, S",NYM,2B,4,2,0,0,0,0,0,...,17,4,5.0,0,0,0,0,0,-,0.00
847,"Jackson, D",LAD,SS,3,0,0,0,0,0,0,...,17,4,8.0,3,3,0,0,0,1.000,1.00
870,"Marrero, D",MIA,3B,5,0,0,0,0,0,0,...,17,5,11.0,4,1,3,0,1,1.000,1.33


In [337]:
# df_total_infld.info()
# df_total_infld.dtypes 데이터 타입만 보려면

### 이렇게 해서 데이터 타입을 바꾸니 문제가 있네
```
# object --> numeric(int or float) 
# 현재 데이터는 전부 object type이여서 수치형 데이터는 pd.to_numeric함수를 이용해 변환
# 데이터프레임에서 컬럼으로 슬라이싱하려면 iloc을 사용
# df_total_infld['G_x'] = pd.to_numeric(df_total_infld['G_x'])
df_total_infld = df_total_infld.apply(pd.to_numeric.iloc, errors='coerce')

# 문자열 타입인데 numeric 타입으로 바뀐것을 원상 복구
# numeric(int or float) --> object
df_total_infld.astype({'Player':'str', 'Team':'str', 'Pos':'str'}).info()
```
- object가 numeric으로 바뀌니깐 NaN값이 되어 버림

In [339]:
col_list = ['AB', 'R', 'H', '2B', '3B', 'HR', 'RBI',
       'BB', 'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'IBB', 'HBP', 'SAC',
       'SF', 'TB', 'XBH', 'GDP', 'GO', 'AO', 'NP', 'PA', 'INN',
       'TC', 'PO', 'A', 'E', 'DP', 'FPCT', 'RF']
for i in col_list:
    df_total_infld[i] = pd.to_numeric(df_total_infld[i], errors='coerce')

C:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [340]:
# 특정 컬럼의 null값이 몇개 있는지 확인
df_total_infld.iloc[:,3:]
print(df_total_infld['FPCT'].isnull().value_counts())

False    286
True       2
Name: FPCT, dtype: int64


In [341]:
# NaN값을 N으로 변경
df_total_infld['FPCT'].fillna('N', inplace=True)
# df_total_infld.info()

## 데이터 분석하기
- groupby를 이용하여 분석하기


In [342]:
df_total_infld

,Player,Team,Pos,AB,R,H,2B,3B,HR,RBI,...,NP,PA,INN,TC,PO,A,E,DP,FPCT,RF
12,"Colon, C",CIN,2B,6,1,3,0,0,0,1,...,28,8,3.2,1,0,1,0,0,1,0.33
22,"Kendrick, H",WSH,3B,334,61,115,23,1,17,62,...,1397,370,601.0,401,308,91,2,35,0.995,4.29
23,"Anderson, T",CWS,SS,498,81,167,32,0,18,56,...,1765,518,1050.0,532,182,324,26,75,0.951,4.11
24,"Arraez, L",MIN,2B,326,54,109,20,1,4,28,...,1486,366,716.0,227,101,121,5,31,0.978,2.22
37,"Solano, D",SF,SS,215,27,71,13,1,4,23,...,903,228,407.0,199,84,113,2,27,0.99,3.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,"Stamets, E",CLE,SS,41,4,2,1,0,0,2,...,192,48,118.0,40,10,28,2,3,0.95,2.53
836,"Haggerty, S",NYM,2B,4,2,0,0,0,0,0,...,17,4,5.0,0,0,0,0,0,N,0.00
847,"Jackson, D",LAD,SS,3,0,0,0,0,0,0,...,17,4,8.0,3,3,0,0,0,1,1.00
870,"Marrero, D",MIA,3B,5,0,0,0,0,0,0,...,17,5,11.0,4,1,3,0,1,1,1.33


In [343]:
# 포지션 별로 각 수치 평균

df_total_infld.groupby('Pos').agg(['mean'])

,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,...,AO,NP,PA,INN,TC,PO,A,E,DP,RF
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
Pos,,,,,,,,,,,,,,,,,,,,,
1B,292.264706,41.838235,72.676471,15.220588,0.764706,14.367647,45.735294,35.573529,78.117647,1.264706,...,74.602941,1357.647059,333.911765,576.789706,509.161765,459.044118,46.000000,4.117647,45.602941,5.274265
2B,253.609756,34.536585,65.487805,12.451220,1.439024,8.292683,30.707317,21.487805,56.012195,4.182927,...,67.000000,1083.280488,280.609756,561.090244,251.963415,109.390244,137.402439,5.170732,33.243902,2.850854
3B,287.960526,41.697368,75.315789,15.842105,1.342105,12.671053,41.855263,28.684211,69.697368,2.644737,...,76.315789,1273.500000,322.763158,626.130263,219.973684,93.039474,120.302632,6.631579,17.618421,2.539474
SS,301.064516,43.112903,80.435484,16.629032,1.854839,11.000000,38.483871,23.451613,66.919355,6.290323,...,77.387097,1263.306452,329.677419,667.182258,291.758065,97.129032,187.096774,7.532258,39.451613,3.071290
